# Processing of SCIP features

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# %load ../common_.py
import sys
sys.path.append("..")
from common import *

In [3]:
import pyarrow.parquet as pq
import pyarrow
from pandas.api.types import CategoricalDtype

## SCIP computed features

In [4]:
data_dir = Path("/home/maximl/scratch/data/CD7/")

In [11]:
data_dir = Path("/data/gent/vo/000/gvo00070/vsc42015/datasets/cd7/800/results/scip/202203221745/")

In [12]:
df = pandas.concat([
    pq.read_table(p).to_pandas()
    for p in Path(data_dir / "").glob("*.parquet")
], axis=0)
df = df.reset_index(drop=True)

In [7]:
df["meta_path"] = df["meta_path"].map(lambda a: "/data/gent/vo/000/gvo00070/vsc42015/datasets/cd7/800/" + os.path.basename(a))

In [8]:
df["meta_P"] = df["meta_scene"].apply(lambda s: int(s.split("-")[0][1:]))
df["meta_replicate"] = df["meta_scene"].apply(lambda s: int(s.split("-")[1][1:]))
df["meta_panel"] = df["meta_scene"].apply(lambda s: s.split("-")[1][0])

In [9]:
cat_type = CategoricalDtype(categories=sorted(df["meta_P"].unique()), ordered=True)
df["meta_P"] = df["meta_P"].astype(cat_type)
cat_type = CategoricalDtype(categories=sorted(df["meta_replicate"].unique()), ordered=True)
df["meta_replicate"] = df["meta_replicate"].astype(cat_type)
cat_type = CategoricalDtype(categories=sorted(df["meta_panel"].unique()), ordered=True)
df["meta_panel"] = df["meta_panel"].astype(cat_type)

In [11]:
pq.write_table(pyarrow.Table.from_pandas(df), data_dir / "features.parquet")

## Features

In [6]:
df = pq.read_table(data_dir / "features.parquet").to_pandas()

In [7]:
from scip.features import shape

In [8]:
shape_cols = df.columns[[any(prop in c for prop in shape.prop_ids) for c in df.columns]]

In [9]:
df = df.drop(columns=[c for c in shape_cols if not c.endswith(("combined", "DAPI"))])

In [12]:
df = df.drop(columns=[c for c in df.columns if "bgcorr" in c])

In [13]:
df.shape

(45942, 836)

In [14]:
numpy.save(data_dir / "indices/columns.npy", df.columns)